In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Rio Grande Do Norte - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [7]:
data = pd.read_csv('2003_01_model_input_RN.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande Do Norte - Consumo de Cimento (t)
0,2003,0.689488,338.269707,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.294170,3.164847e+07,1.998664e+06,9.097985,2.879532e+07,408.265000
1,2004,0.694345,350.872937,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.222820,3.213445e+07,2.027983e+06,9.137415,2.897550e+07,404.604000
2,2005,0.698728,368.606856,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.202615,3.257756e+07,2.041700e+06,9.167875,2.911598e+07,450.141000
3,2006,0.702649,400.635463,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.110415,3.296767e+07,2.059520e+06,9.185353,2.921015e+07,526.789000
4,2007,0.706119,425.000411,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.087322,3.330228e+07,2.079730e+06,9.191188,2.926087e+07,602.047000
5,2008,0.709150,443.220730,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.054622,3.355987e+07,2.079113e+06,9.184161,2.924771e+07,705.791000
6,2009,0.711755,467.794411,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.084471,3.376729e+07,2.091124e+06,9.172749,2.923115e+07,694.478000
7,2010,0.713945,615.433124,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.047148,3.462025e+07,2.094395e+06,9.216787,2.947144e+07,850.190000
8,2011,0.715731,928.296392,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,8.031985,3.538151e+07,2.094876e+06,9.249340,2.964338e+07,836.975000
9,2012,0.716652,1212.359182,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,12.625418,3.604796e+07,2.083213e+06,9.259759,2.976380e+07,961.431000


In [8]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado
0,0.689488,338.269707,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.294170,3.164847e+07,1.998664e+06,9.097985,2.879532e+07
1,0.694345,350.872937,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.222820,3.213445e+07,2.027983e+06,9.137415,2.897550e+07
2,0.698728,368.606856,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.202615,3.257756e+07,2.041700e+06,9.167875,2.911598e+07
3,0.702649,400.635463,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.110415,3.296767e+07,2.059520e+06,9.185353,2.921015e+07
4,0.706119,425.000411,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.087322,3.330228e+07,2.079730e+06,9.191188,2.926087e+07
5,0.709150,443.220730,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.054622,3.355987e+07,2.079113e+06,9.184161,2.924771e+07
6,0.711755,467.794411,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.084471,3.376729e+07,2.091124e+06,9.172749,2.923115e+07
7,0.713945,615.433124,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.047148,3.462025e+07,2.094395e+06,9.216787,2.947144e+07
8,0.715731,928.296392,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,8.031985,3.538151e+07,2.094876e+06,9.249340,2.964338e+07
9,0.716652,1212.359182,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,12.625418,3.604796e+07,2.083213e+06,9.259759,2.976380e+07


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     404.604000
1     450.141000
2     526.789000
3     602.047000
4     705.791000
5     694.478000
6     850.190000
7     836.975000
8     961.431000
9     954.170000
10    886.165714
11    802.508571
12    730.261714
13    646.482000
14    485.012000
15    563.293000
16    744.863000
17    811.860000
18    866.416000
19           NaN
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado
0,-2.030201,-1.232653,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-0.872684,-1.665069,-0.692195,-2.494629,-2.054569
1,-1.619870,-1.198579,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-0.901222,-1.430595,-0.112443,-1.601249,-1.523255
2,-1.249614,-1.150633,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-0.909304,-1.216803,0.158793,-0.911104,-1.109036
3,-0.918427,-1.064039,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.946181,-1.028589,0.511163,-0.515102,-0.831350
4,-0.625303,-0.998165,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.955417,-0.867143,0.910789,-0.382880,-0.681812
5,-0.369238,-0.948904,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.968496,-0.742862,0.898592,-0.542103,-0.720604
6,-0.149226,-0.882465,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.956558,-0.642789,1.136098,-0.800667,-0.769444
7,0.035737,-0.483304,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.971486,-0.231255,1.200775,0.197129,-0.060899
8,0.186658,0.362565,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-0.977551,0.136034,1.210287,0.934690,0.446116
9,0.264396,1.130569,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.859698,0.457580,0.979680,1.170769,0.801192


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     404.604000
1     450.141000
2     526.789000
3     602.047000
4     705.791000
5     694.478000
6     850.190000
7     836.975000
8     961.431000
9     954.170000
10    886.165714
11    802.508571
12    730.261714
13    646.482000
14    485.012000
15    563.293000
16    744.863000
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado
17,1.048153,0.793617,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.939491,0.751768,-1.131139,-0.273722,0.314748


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    811.86
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [14]:
train_input.iloc[-(len(train_input)//5):]

,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado
14,1.293744,0.512794,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,1.063470,1.202019,-1.545731,0.805914,1.026058
15,1.300173,0.362687,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,1.030632,1.152827,-1.546308,0.673637,0.883841
16,1.222715,0.548932,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,1.023618,1.030559,-1.412843,0.341727,0.696252


In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[4216193802, 3763108227, 860493056, 1020057834, 1331048516, 3556448978, 4264261582, 722867637, 1081889940, 103819882, 3640902033, 3471655523, 1164408831, 599481908, 3135174752, 2814638918, 682669183, 2534202617, 1359460816, 4020990360, 3565283705, 4108753357, 3825269026, 963647744, 1378713191, 3605524336, 3351535963, 240931333, 715244225, 705403041, 833433007, 2262871039, 4076626909, 1936520598, 1938485652, 3135798019, 3721168403, 711049208, 1413930061, 3367212460, 3938606024, 381044390, 3521853210, 577814026, 3586149334, 1898152397, 2928326292, 1949743371, 1763662534, 10304543, 1057377455, 3353157260, 3534956349, 817215134, 164809763, 24357281, 823458436, 2066644162, 523636817, 442703018, 2362801769, 4225843424, 1973414345, 2999565487, 2827024768, 2533507172, 2052320827, 3471477610, 3238327337, 1633751038, 3157815266, 2719228801, 3473863293, 889595827, 3886515183, 283521042, 1915712970, 997824974, 1574872631, 3800977170, 3551122582, 3914773045, 2517092804, 1075190024, 2611108726, 3388

2023-10-02 18:45:29.000010: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-02 18:45:29.018433: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 13193.6826171875


Step: 53 ___________________________________________
val_loss: 26231.546875


Step: 54 ___________________________________________
val_loss: 11857.9150390625


Step: 55 ___________________________________________
val_loss: 11861.4990234375


Step: 56 ___________________________________________
val_loss: 14903.8623046875


Step: 57 ___________________________________________
val_loss: 27213.21875


Step: 58 ___________________________________________
val_loss: 27127.314453125


Step: 59 ___________________________________________
val_loss: 12577.3671875


Step: 60 ___________________________________________
val_loss: 17663.787109375


Step: 61 ___________________________________________
val_loss: 27055.693359375


Step: 62 ___________________________________________
val_loss: 33920.8203125


Step: 63 ___________________________________________
val_loss: 16850.634765625


Step: 64 ___________________________________________
val_loss: 30205.5234375


Step: 65 

2023-10-02 18:50:12.961091: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-02 18:50:12.997539: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 23045.939453125


Step: 71 ___________________________________________
val_loss: 12428.5595703125


Step: 72 ___________________________________________
val_loss: 14407.4228515625


Step: 73 ___________________________________________
val_loss: 28817.78125


Step: 74 ___________________________________________
val_loss: 13746.5185546875


Step: 75 ___________________________________________
val_loss: 18176.849609375


Step: 76 ___________________________________________
val_loss: 21574.4765625


Step: 77 ___________________________________________
val_loss: 25622.841796875


Step: 78 ___________________________________________
val_loss: 12304.7421875


Step: 79 ___________________________________________
val_loss: 10308.7470703125


Step: 80 ___________________________________________
val_loss: 23038.630859375


Step: 81 ___________________________________________
val_loss: 11891.4765625


Step: 82 ___________________________________________
val_loss: 11892.3427734375


Step:

In [19]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 449ms/step - loss: 513641.2812 - val_loss: 341648.2812
Epoch 2/10000
1/1 [==============================] - 0s 13ms/step - loss: 488921.4688 - val_loss: 113124.6172
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 541989.5000 - val_loss: 327669.9688
Epoch 4/10000
1/1 [==============================] - 0s 12ms/step - loss: 481846.9375 - val_loss: 346943.8438
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 299491.8125 - val_loss: 376147.3750
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 270897.3750 - val_loss: 375369.6562
Epoch 7/10000
1/1 [==============================] - 0s 12ms/step - loss: 275161.7500 - val_loss: 119241.1016
Epoch 8/10000
1/1 [==============================] - 0s 12ms/step - loss: 288048.2812 - val_loss: 150650.7031
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 327819.8750 - val_loss: 380116.2812
Epoch 10/

1/1 [==============================] - 0s 12ms/step - loss: 104520.3281 - val_loss: 231455.8594
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 108927.5625 - val_loss: 213086.2344
Epoch 77/10000
1/1 [==============================] - 0s 12ms/step - loss: 121574.7969 - val_loss: 191555.4531
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 100093.3047 - val_loss: 78709.9844
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 93881.5547 - val_loss: 59688.2148
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 88530.9766 - val_loss: 57835.9805
Epoch 81/10000
1/1 [==============================] - 0s 12ms/step - loss: 87952.1250 - val_loss: 47831.6289
Epoch 82/10000
1/1 [==============================] - 0s 12ms/step - loss: 116135.9453 - val_loss: 37237.5977
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 80232.7969 - val_loss: 45158.3320
Epoch 84/10000
1/1 [======

1/1 [==============================] - 0s 13ms/step - loss: 46226.1133 - val_loss: 59238.7383
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 39046.4375 - val_loss: 59436.2383
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 34051.2773 - val_loss: 61974.6133
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 38274.0117 - val_loss: 54024.5273
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 38034.8789 - val_loss: 58976.9648
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 37219.5039 - val_loss: 55412.6562
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 36770.1719 - val_loss: 54935.3594
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 36333.1562 - val_loss: 54011.2852
Epoch 157/10000
1/1 [==============================] - 0s 13ms/step - loss: 42133.7148 - val_loss: 49765.2031
Epoch 158/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 25744.9590 - val_loss: 54593.5000
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 25587.8242 - val_loss: 54176.9180
Epoch 225/10000
1/1 [==============================] - 0s 12ms/step - loss: 25417.1797 - val_loss: 53843.7227
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 22575.8672 - val_loss: 41320.0977
Epoch 227/10000
1/1 [==============================] - 0s 13ms/step - loss: 22725.2656 - val_loss: 98996.3359
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 25984.4355 - val_loss: 87564.1484
Epoch 229/10000
1/1 [==============================] - 0s 12ms/step - loss: 25537.5605 - val_loss: 65253.4023
Epoch 230/10000
1/1 [==============================] - 0s 13ms/step - loss: 28788.1621 - val_loss: 68449.7344
Epoch 231/10000
1/1 [==============================] - 0s 13ms/step - loss: 25860.9551 - val_loss: 60229.2930
Epoch 232/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 17096.6035 - val_loss: 43333.1992
Epoch 298/10000
1/1 [==============================] - 0s 13ms/step - loss: 18001.1621 - val_loss: 46333.1367
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 16951.0469 - val_loss: 42870.1602
Epoch 300/10000
1/1 [==============================] - 0s 13ms/step - loss: 17476.6211 - val_loss: 40621.0742
Epoch 301/10000
1/1 [==============================] - 0s 13ms/step - loss: 17860.8320 - val_loss: 46082.8125
Epoch 302/10000
1/1 [==============================] - 0s 13ms/step - loss: 17655.5312 - val_loss: 44951.3750
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 17237.8535 - val_loss: 45328.8945
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 16589.5449 - val_loss: 41972.2188
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 17481.4980 - val_loss: 44963.3086
Epoch 306/10000
1/1 [=====

...

1/1 [==============================] - 0s 13ms/step - loss: 13584.3047 - val_loss: 30017.0605
Epoch 1045/10000
1/1 [==============================] - 0s 13ms/step - loss: 13820.9668 - val_loss: 29280.0215
Epoch 1046/10000
1/1 [==============================] - 0s 13ms/step - loss: 13788.8506 - val_loss: 30555.1738
Epoch 1047/10000
1/1 [==============================] - 0s 13ms/step - loss: 13514.2871 - val_loss: 30835.3594
Epoch 1048/10000
1/1 [==============================] - 0s 13ms/step - loss: 13772.0703 - val_loss: 30184.9473
Epoch 1049/10000
1/1 [==============================] - 0s 12ms/step - loss: 13766.2256 - val_loss: 30290.5449
Epoch 1050/10000
1/1 [==============================] - 0s 13ms/step - loss: 13763.2900 - val_loss: 30250.7520
Epoch 1051/10000
1/1 [==============================] - 0s 13ms/step - loss: 13812.4990 - val_loss: 29084.2559
Epoch 1052/10000
1/1 [==============================] - 0s 13ms/step - loss: 13548.1318 - val_loss: 29820.6582
Epoch 1053/10000
1

Epoch 1118/10000
1/1 [==============================] - 0s 12ms/step - loss: 13817.1621 - val_loss: 31143.2520
Epoch 1119/10000
1/1 [==============================] - 0s 13ms/step - loss: 13785.8281 - val_loss: 30711.0840
Epoch 1120/10000
1/1 [==============================] - 0s 12ms/step - loss: 13872.0508 - val_loss: 31966.8066
Epoch 1121/10000
1/1 [==============================] - 0s 13ms/step - loss: 12796.5156 - val_loss: 31633.0098
Epoch 1122/10000
1/1 [==============================] - 0s 13ms/step - loss: 13850.8926 - val_loss: 31630.7832
Epoch 1123/10000
1/1 [==============================] - 0s 12ms/step - loss: 13419.4492 - val_loss: 32846.1914
Epoch 1124/10000
1/1 [==============================] - 0s 12ms/step - loss: 13819.6787 - val_loss: 30008.7832
Epoch 1125/10000
1/1 [==============================] - 0s 12ms/step - loss: 13865.7139 - val_loss: 31744.3906
Epoch 1126/10000
1/1 [==============================] - 0s 12ms/step - loss: 13771.5986 - val_loss: 30058.1562
E

Epoch 1192/10000
1/1 [==============================] - 0s 13ms/step - loss: 12438.7256 - val_loss: 30087.0156
Epoch 1193/10000
1/1 [==============================] - 0s 12ms/step - loss: 13314.0303 - val_loss: 29362.4160
Epoch 1194/10000
1/1 [==============================] - 0s 13ms/step - loss: 18967.9219 - val_loss: 34311.5000
Epoch 1195/10000
1/1 [==============================] - 0s 12ms/step - loss: 9962.3594 - val_loss: 19660.3672
Epoch 1196/10000
1/1 [==============================] - 0s 12ms/step - loss: 9880.5254 - val_loss: 19376.8965
Epoch 1197/10000
1/1 [==============================] - 0s 12ms/step - loss: 9938.4121 - val_loss: 20700.0020
Epoch 1198/10000
1/1 [==============================] - 0s 13ms/step - loss: 8222.0234 - val_loss: 23248.9062
Epoch 1199/10000
1/1 [==============================] - 0s 13ms/step - loss: 10155.2578 - val_loss: 20397.9727
Epoch 1200/10000
1/1 [==============================] - 0s 13ms/step - loss: 10510.9629 - val_loss: 20787.4160
Epoch

1/1 [==============================] - 0s 13ms/step - loss: 8050.8179 - val_loss: 20137.6973
Epoch 1267/10000
1/1 [==============================] - 0s 13ms/step - loss: 5629.9414 - val_loss: 22189.2520
Epoch 1268/10000
1/1 [==============================] - 0s 12ms/step - loss: 8695.7578 - val_loss: 16743.0098
Epoch 1269/10000
1/1 [==============================] - 0s 12ms/step - loss: 8338.2646 - val_loss: 21862.8184
Epoch 1270/10000
1/1 [==============================] - 0s 12ms/step - loss: 7059.4487 - val_loss: 22163.8066
Epoch 1271/10000
1/1 [==============================] - 0s 12ms/step - loss: 5825.6406 - val_loss: 19521.0977
Epoch 1272/10000
1/1 [==============================] - 0s 13ms/step - loss: 5768.8750 - val_loss: 19467.9316
Epoch 1273/10000
1/1 [==============================] - 0s 13ms/step - loss: 7278.7246 - val_loss: 20694.5957
Epoch 1274/10000
1/1 [==============================] - 0s 12ms/step - loss: 4983.5459 - val_loss: 20497.6777
Epoch 1275/10000
1/1 [=====

Epoch 1341/10000
1/1 [==============================] - 0s 13ms/step - loss: 8385.5986 - val_loss: 16306.3359
Epoch 1342/10000
1/1 [==============================] - 0s 13ms/step - loss: 7235.0752 - val_loss: 13706.6484
Epoch 1343/10000
1/1 [==============================] - 0s 13ms/step - loss: 7787.8950 - val_loss: 13636.4834
Epoch 1344/10000
1/1 [==============================] - 0s 12ms/step - loss: 6765.7534 - val_loss: 13208.5498
Epoch 1345/10000
1/1 [==============================] - 0s 13ms/step - loss: 7058.9619 - val_loss: 13695.4775
Epoch 1346/10000
1/1 [==============================] - 0s 12ms/step - loss: 6463.7480 - val_loss: 14565.5420
Epoch 1347/10000
1/1 [==============================] - 0s 13ms/step - loss: 6744.7217 - val_loss: 13042.3779
Epoch 1348/10000
1/1 [==============================] - 0s 13ms/step - loss: 6698.3555 - val_loss: 11439.4648
Epoch 1349/10000
1/1 [==============================] - 0s 13ms/step - loss: 7155.4282 - val_loss: 14207.8086
Epoch 1350

In [20]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 46ms/step
[811.86](test_target) - [[759.1169]](prediction) = 52.743117675781264


In [21]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [22]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado
0,-1.711016,-1.058842,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,1.848182,-1.600293,-1.906391,-2.085127,-1.929514
1,-1.108077,-0.907127,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,1.004081,-1.054056,-0.980456,-0.930471,-0.985258
2,-0.564023,-0.693649,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,0.765040,-0.556001,-0.547259,-0.038487,-0.249104
3,-0.077377,-0.308093,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.325730,-0.117531,0.015518,0.473329,0.244404
4,0.353338,-0.014792,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.598927,0.258577,0.653770,0.644220,0.510165
5,0.729600,0.204541,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,-0.985782,0.548105,0.634290,0.438431,0.441223
6,1.052885,0.500355,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,-0.632652,0.781240,1.013615,0.104248,0.354425
7,1.324670,2.277607,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,-1.074212,1.739960,1.116913,1.393857,1.613659


train_target:


,Rio Grande Do Norte - Consumo de Cimento (t)
0,404.604
1,450.141
2,526.789
3,602.047
4,705.791
5,694.478
6,850.190
7,836.975


test_input:


...

,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado
17,1.048153,0.793617,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.939491,0.751768,-1.131139,-0.273722,0.314748


test_target:


,Rio Grande Do Norte - Consumo de Cimento (t)
17,811.86


1/1 [==============================] - 0s 34ms/step
Error: 52.743117675781264




[793.9609375,
 839.8878784179688,
 918.4461669921875,
 811.4039916992188,
 844.3372802734375,
 855.5416259765625,
 702.2365112304688,
 692.533447265625,
 652.7637939453125,
 759.1168823242188]

In [23]:
display(targets)
display(predictions)

[961.431,
 954.17,
 886.1657142857142,
 802.5085714285714,
 730.2617142857143,
 646.482,
 485.012,
 563.293,
 744.8629999999999,
 811.86]

[793.9609375,
 839.8878784179688,
 918.4461669921875,
 811.4039916992188,
 844.3372802734375,
 855.5416259765625,
 702.2365112304688,
 692.533447265625,
 652.7637939453125,
 759.1168823242188]

In [24]:
mae = mean_absolute_error(predictions, targets)
mae

113.73705312500002

In [25]:
porcentage = mae/np.mean(targets)
porcentage

0.1499292755831858